# Litepose 
[Litepose][1] proposes an efficient way to perform multi-person pose estimation, providing a low computationl cost, scale invariant and reliable architecture. It follows a bottom-up approach, namely it uses just one network to do both keypoints estimation and grouping. In this implementation I have also relied on two other papers (also cited by Litepose), one is [HigherHRNet][2] that proposes the main architecture and the other is [Associative Embedding][3] that introduces a way to assign identity-free keypoints to the person they belong to.

Litepose modifies the HigherHRNet architecture going from a multi-branch to a single-branch one by gradual shrinking, with the purpose of speeding up the inference, making it run on low computational power devices as well.

The architecture uses a MobileNet backbone with **Large Kernel Convolutions** that have shown great results empirically. The backbone output is passed to multiple deconvolutional blocks implementing the main feature of the Litepose Paper that is **Fusion Deconv Head**. This allows to obtain scale aware results by merging backbone intermediate features and refined features, in this way the network can exploit high resolution features, that help to catch close joints, without involving a multi-branch architecture. 

Let $t$ be the number of convolutional blocks and $n$ be the number of the current deconvolutional block, the features fusion is implemented by summing the features of deconvolutional block in position $n$ with the features of backbone block in position $t-n-1$, refined by an additional convolutional layer. Eventually the merged features are passed to a final block for each deconvolutional layer that produces the output. The results of the network are provided in several scales, one for each deconvolutional layer. Hence the output is a $(n,j,s_i,s_i)$ tensor where $n$ is the number of scales (i.e. deconv blocks), $j$ is the number of joints that we want to detect, $s_i$ for $i\in{1,2,...,n}$ is the size of the current scale. The image clarifies the network structure.

![Network Architecture!](../assets/structure.png)


[1]:https://openaccess.thecvf.com/content/CVPR2022/papers/Wang_Lite_Pose_Efficient_Architecture_Design_for_2D_Human_Pose_Estimation_CVPR_2022_paper.pdf
[2]:https://arxiv.org/pdf/1908.10357.pdf
[3]:https://papers.nips.cc/paper/2017/file/8edd72158ccd2a879f79cb2538568fdc-Paper.pdf

In [1]:
from lp_coco_utils.lp_getDataset import getDatasetProcessed
from lp_training.lp_trainer import train
from lp_model.lp_litepose import LitePose
import lp_config.lp_common_config as cc
import torch
from lp_inference.lp_inference import inference, assocEmbedding
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, normalizeImage, drawSkeleton
from lp_testing.lp_evaluation import computeOKS

This file has to be seen only as an entry that calls wrapper functions, the implentation of those functions can be found in the subdirectories of the repository.   
Every hyperparameter can be edited in `src/lp_config`.  
`lp_common_config.py` contains the general configurations about the dataset loading, training and inference. On the other hand `lp_model_config.py` contrains the parameters that encode the model structure. The current model configs are taken from the Neural Architecture Search performed by the paper authors. I used the small size network due to the computational power available, however better results can be achieved simply by scaling the network size (Good parameters combinations are provided by the paper authors).

Code taken by the [official paper repository](https://github.com/mit-han-lab/litepose):
- classes `CocoDataset` and `CocoKeypoints` are partially taken, I added fiftyone support that makes the dataset setup easier and I removed unnecessary code.
- I took the code inside `lp_generators.py` and `lp_transforms.py` as well, since they were a `CocoKeypoints` dependencies

# Training

The dataset is downloaded by using fiftyone APIs and keypoint heatmaps are created for each sample. 

In [2]:
train(cc.config["batch_size"])

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


100%|██████████| 250/250 [00:23<00:00, 10.84it/s]


epoch #1 

TRAINING LOSS:
total Loss = 0.1084165603145957
heatmap Loss = 0.10788685510307551
tag Loss = 0.0005297049182699993

VALIDATION LOSS:
total Loss = 0.04165456732362509
heatmap Loss = 0.041182924181222914
tag Loss = 0.00047164313518442214





100%|██████████| 250/250 [00:23<00:00, 10.55it/s]


epoch #2 

TRAINING LOSS:
total Loss = 0.022915598655119537
heatmap Loss = 0.02244729833677411
tag Loss = 0.0004683002809761092

VALIDATION LOSS:
total Loss = 0.012833793051540851
heatmap Loss = 0.012369948850944638
tag Loss = 0.00046384419675450775





...

epoch #196 

TRAINING LOSS:
total Loss = 0.005860954756848514
heatmap Loss = 0.005591240432113409
tag Loss = 0.000269714318739716

VALIDATION LOSS:
total Loss = 0.005830740993842482
heatmap Loss = 0.0055620934236794714
tag Loss = 0.00026864757086150347





100%|██████████| 250/250 [00:23<00:00, 10.78it/s]


epoch #197 

TRAINING LOSS:
total Loss = 0.005821276003494859
heatmap Loss = 0.005552753589116037
tag Loss = 0.00026852240378502754

VALIDATION LOSS:
total Loss = 0.00588785233348608
heatmap Loss = 0.005619577812962234
tag Loss = 0.0002682745179918129





100%|██████████| 250/250 [00:23<00:00, 10.86it/s]


epoch #198 

TRAINING LOSS:
total Loss = 0.005886817715130746
heatmap Loss = 0.005616611158475279
tag Loss = 0.00027020656672539187

VALIDATION LOSS:
total Loss = 0.005825847988948226
heatmap Loss = 0.0055597203979268674
tag Loss = 0.00026612760324496777



end training, exec time: 11896.716822385788


# Inference
Unfotunately OpenCV method `imshow()` has a well known bug with python notebooks.

In [2]:
import cv2
import random

import torch.nn.functional as F


model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load("lp_trained_models/definitive"))

ds = getDatasetProcessed("validation")

data_loader = torch.utils.data.DataLoader(
    ds,
    batch_size=8
)
it = iter(data_loader)
next(it)
next(it)

row = next(it)
images = row[0].to(cc.config["device"])

gthm = row[1]
output, keypoints = inference(model, images)

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [14]:
embedding = assocEmbedding(keypoints)

In [3]:
jointsHeatmap = output[1][2][:cc.config["num_joints"]]

img, finalHm, superimposed = drawHeatmap(images[2], jointsHeatmap)
img, gtfinalHm, gtsuperimposed = drawHeatmap(images[2], gthm[1][2])
cv2.imshow("Image", img)
cv2.imshow("Final heatmap", finalHm)
cv2.imshow("Superimposed", superimposed)

cv2.imshow("Ground Truth heatmap", gtfinalHm)
cv2.imshow("Ground Truth Superimposed", gtsuperimposed)
cv2.waitKey()
cv2.destroyAllWindows()

In [15]:
import lp_utils.lp_image_processing as ip
import numpy as np

img = images[1]

tj = output[1][1]

for t in range(14,28):
    tagJoints = tj[t]
    scaled = ip.scaleImage(tagJoints.unsqueeze(0), img.shape[1]).cpu().numpy()
    scaled = scaled[0]
    scaled = ip.normalizeImage(scaled)
    finalHm = cv2.applyColorMap(np.uint8(scaled), cv2.COLORMAP_JET)
    cv2.imshow(str(t), finalHm)
    cv2.waitKey()
cv2.destroyAllWindows()


In [3]:
tagJoints = output[1][2][cc.config["num_joints"]:]

print(tagJoints[0].max())

img, finalHm, superimposed = drawHeatmap(images[2], tagJoints)
cv2.imshow("Image", img)
cv2.imshow("Final heatmap", finalHm)
cv2.imshow("Superimposed", superimposed)

cv2.waitKey()
cv2.destroyAllWindows()

tensor(0., device='cuda:0')


In [16]:
img = drawKeypoints(images[5], keypoints[5])
cv2.imshow("Image Keypoints", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [25]:
idx = 5
img = drawSkeleton(images[idx], embedding[idx])
cv2.imshow("Image Keypoints", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
import cv2 
import torch
import numpy as np
from lp_config.lp_common_config import config

@torch.no_grad()
def keypointOnCam(model, savePath, confidenceThreshold = 0):
    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
    if (cap.isOpened() == False):
        print('Error while trying to open webcam')
        return
    
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    out = cv2.VideoWriter(f"{savePath}", 
                      cv2.VideoWriter_fourcc(*'mp4v'), 20, 
                      (frame_width, frame_height))

    while(cap.isOpened()):
    # capture each frame of the video
        ret, frame = cap.read()
        if ret == True:

            image = frame
            image = cv2.resize(image, (256, 256))
            orig_frame = image.copy()
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image / 255.0
            image = np.transpose(image, (2, 0, 1))
            image = torch.tensor(image, dtype=torch.float)
            image = image.unsqueeze(0).to(config["device"])
            kps = inference(model, image)
            skel = assocEmbedding(kps)

            orig_frame = drawSkeleton(orig_frame, skel)
            
            orig_frame = cv2.resize(orig_frame, (frame_width, frame_height))
            cv2.imshow('Keypoint Frame', orig_frame)
            out.write(orig_frame)

            if cv2.waitKey(27) & 0xFF == ord('q'):
                break
    
        else: 
            break
        
    cap.release()
    cv2.destroyAllWindows()



In [4]:
keypointOnCam(model.to(device=config["device"]),"ciao.mp4")

IndexError: list index out of range